In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
import joblib

In [17]:
data = pd.read_csv('data_balita.csv')
data.head()

,Umur (bulan),Jenis Kelamin,Tinggi Badan (cm),Status Gizi
0,0,laki-laki,44.591973,stunted
1,0,laki-laki,56.705203,tinggi
2,0,laki-laki,46.863358,normal
3,0,laki-laki,47.508026,normal
4,0,laki-laki,42.743494,severely stunted


In [18]:
#mengkodekan jenis kelamin
data['Jenis Kelamin'] = data['Jenis Kelamin'].map({'laki-laki' : 0, 'perempuan' : 1})

#mengkodekan status gizi
data['Status Gizi'] = data['Status Gizi'].map({'severely stunted' : 0, 'stunted' : 1, 'normal' : 2, 'tinggi' : 3})
data.head()

,Umur (bulan),Jenis Kelamin,Tinggi Badan (cm),Status Gizi
0,0,0,44.591973,1
1,0,0,56.705203,3
2,0,0,46.863358,2
3,0,0,47.508026,2
4,0,0,42.743494,0


In [19]:
#memisahkan fitur dan target
X = data.drop('Status Gizi', axis = 1)
y = data['Status Gizi']

#membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X.head()

,Umur (bulan),Jenis Kelamin,Tinggi Badan (cm)
0,0,0,44.591973
1,0,0,56.705203
2,0,0,46.863358
3,0,0,47.508026
4,0,0,42.743494


In [20]:
model_ann = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # Lapisan input
    layers.Dense(64, activation='relu'),  # Lapisan tersembunyi
    layers.Dense(4, activation='softmax')  # Lapisan output untuk 4 kelas
])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model_ann.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [22]:
model_ann.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
2420/2420 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - accuracy: 0.5497 - loss: 1.1702 - val_accuracy: 0.6792 - val_loss: 0.8361
Epoch 2/50
2420/2420 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - accuracy: 0.7088 - loss: 0.7192 - val_accuracy: 0.8794 - val_loss: 0.3754
Epoch 3/50
2420/2420 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - accuracy: 0.8613 - loss: 0.3528 - val_accuracy: 0.9168 - val_loss: 0.2362
Epoch 4/50
2420/2420 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - accuracy: 0.9078 - loss: 0.2436 - val_accuracy: 0.9638 - val_loss: 0.1635
Epoch 5/50
2420/2420 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - accuracy: 0.9258 - loss: 0.1978 - val_accuracy: 0.9544 - val_loss: 0.1449
Epoch 6/50
2420/2420 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - accuracy: 0.9251 - loss: 0.1821 - val_accuracy: 0.9662 - val_loss: 0.1219
Epoch 7/50
2420/2420 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - accuracy: 0.9321 - loss: 0.1670 - val_accuracy: 0.9711 - val_loss: 0.1116
Epoch 8/50
2420/2420 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - accuracy: 0.9308 - loss: 0

In [23]:
loss, accuracy = model_ann.evaluate(X_test, y_test)
print(f"Akurasi model ANN: {accuracy * 100:.2f}%")

757/757 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9736 - loss: 0.0694
Akurasi model ANN: 97.17%


In [24]:
y_pred_ann = model_ann.predict(X_test)
y_pred_ann_classes = np.argmax(y_pred_ann, axis=1)

757/757 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step


In [25]:
print(classification_report(y_test, y_pred_ann_classes))

              precision    recall  f1-score   support

           0       1.00      0.93      0.96      4130
           1       0.90      0.93      0.91      2790
           2       0.99      0.98      0.99     13382
           3       0.95      1.00      0.98      3898

    accuracy                           0.97     24200
   macro avg       0.96      0.96      0.96     24200
weighted avg       0.97      0.97      0.97     24200



In [27]:
model_ann.save('stunting_model.h5')